In [34]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
datos = pd.read_excel("02.Generación Pearson Photovoltaic.xlsx")
datos.head(4)

,Fecha,Generación,Índice UV,Temperatura,Hora,DPT,Cobertura Nubes,Condición Cielo,Humedad Relativa
0,2022-09-01 00:00:00,0.0,0.0,19,24,15,98,4,77
1,2022-09-01 01:00:00,0.0,0.0,19,1,16,82,4,82
2,2022-09-01 02:00:00,0.0,0.0,18,2,16,73,3,85
3,2022-09-01 03:00:00,0.0,0.0,18,3,16,63,3,87


In [35]:
datos_2 = datos.copy()
datos_2["Generacion_prev_hour"] = datos_2["Generación"].shift(1)
datos_dropna = datos_2.dropna(how="any", axis= 0)

In [36]:
# Filtrar solo las filas donde la hora esté entre 6:00 y 19:00
datos_filtrados = datos_dropna[(datos_dropna['Fecha'].dt.hour >= 6) & (datos_dropna['Fecha'].dt.hour < 19)]
datos_filtrados

,Fecha,Generación,Índice UV,Temperatura,Hora,DPT,Cobertura Nubes,Condición Cielo,Humedad Relativa,Generacion_prev_hour
6,2022-09-01 06:00:00,0.000000,0.0,18,6,16,54,3,89,0.000000
7,2022-09-01 07:00:00,6.584959,0.0,18,7,17,70,4,95,0.000000
8,2022-09-01 08:00:00,560.422022,0.0,18,8,18,72,3,100,6.584959
9,2022-09-01 09:00:00,7720.582326,1.0,18,9,18,81,4,100,560.422022
10,2022-09-01 10:00:00,9433.109309,2.0,19,10,18,89,4,93,7720.582326
...,...,...,...,...,...,...,...,...,...,...
18230,2024-10-02 14:00:00,25399.000000,8.0,24,14,9,60,3,38,25598.000000
18231,2024-10-02 15:00:00,25562.000000,5.0,25,15,8,43,3,34,25399.000000
18232,2024-10-02 16:00:00,25386.000000,4.0,26,16,8,25,2,31,25562.000000
18233,2024-10-02 17:00:00,22872.000000,2.0,26,17,8,18,1,32,25386.000000


In [37]:
datos_sinfecha = datos_filtrados.drop(["Fecha"], axis=1)

In [38]:
datos_sinfecha.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9880 entries, 6 to 18234
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Generación            9880 non-null   float64
 1   Índice UV             9880 non-null   float64
 2   Temperatura           9880 non-null   int64  
 3   Hora                  9880 non-null   int64  
 4   DPT                   9880 non-null   int64  
 5   Cobertura Nubes       9880 non-null   int64  
 6   Condición Cielo       9880 non-null   int64  
 7   Humedad Relativa      9880 non-null   int64  
 8   Generacion_prev_hour  9880 non-null   float64
dtypes: float64(3), int64(6)
memory usage: 771.9 KB


In [39]:
datos_sinfecha.describe().T

,count,mean,std,min,25%,50%,75%,max
Generación,9880.0,16964.042023,10578.321772,0.0,7407.75,20480.5,26617.25,30000.0
Índice UV,9880.0,3.629676,3.569451,0.0,1.00,3.0,5.00,14.0
Temperatura,9880.0,20.668421,7.293368,0.0,16.00,21.0,26.00,39.0
Hora,9880.0,12.000000,3.741847,6.0,9.00,12.0,15.00,18.0
DPT,9880.0,9.299089,4.772963,0.0,5.00,10.0,13.00,19.0
Cobertura Nubes,9880.0,49.449190,37.137358,0.0,13.00,46.0,89.00,100.0
Condición Cielo,9880.0,2.578846,1.449860,1.0,1.00,2.0,4.00,5.0
Humedad Relativa,9880.0,53.868016,27.426058,3.0,30.00,52.0,80.00,100.0
Generacion_prev_hour,9880.0,15786.879860,11332.952541,0.0,1642.00,19448.0,26586.00,30000.0


In [40]:
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import StandardScaler

# Excluir la columna "Generación"
X = datos_sinfecha.drop(columns=["Generación"])

# Estandarizar los datos para mejorar la estabilidad de GMM
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Definir el número de clusters (puedes ajustar el valor de n_components)
n_clusters = 3  # Puedes probar con otros valores
gmm = GaussianMixture(n_components=n_clusters, random_state=42)

# Ajustar el modelo y predecir los clusters
datos_sinfecha["Cluster"] = gmm.fit_predict(X)

# Mostrar los primeros registros con los clusters asignados
datos_sinfecha.head()


,Generación,Índice UV,Temperatura,Hora,DPT,Cobertura Nubes,Condición Cielo,Humedad Relativa,Generacion_prev_hour,Cluster
6,0.000000,0.0,18,6,16,54,3,89,0.000000,1
7,6.584959,0.0,18,7,17,70,4,95,0.000000,1
8,560.422022,0.0,18,8,18,72,3,100,6.584959,2
9,7720.582326,1.0,18,9,18,81,4,100,560.422022,2
10,9433.109309,2.0,19,10,18,89,4,93,7720.582326,2


In [41]:
datos_sinfecha['Cluster'].value_counts()

Cluster
0    5446
2    2633
1    1801
Name: count, dtype: int64

In [42]:
# Calcular estadísticas de cada cluster
cluster_means = datos_sinfecha.groupby('Cluster').mean()
cluster_means


,Generación,Índice UV,Temperatura,Hora,DPT,Cobertura Nubes,Condición Cielo,Humedad Relativa,Generacion_prev_hour
Cluster,,,,,,,,,
0,22151.563115,5.281069,25.506059,14.785714,8.069960,42.039846,2.239258,33.838964,23020.917866
1,509.498045,0.049639,12.806219,6.736813,10.413104,60.383676,3.133815,85.276513,0.000000
2,17489.447963,2.662780,16.040258,9.838207,11.079377,57.295101,2.901633,73.811622,11622.656406


In [43]:
datos_sinfecha

,Generación,Índice UV,Temperatura,Hora,DPT,Cobertura Nubes,Condición Cielo,Humedad Relativa,Generacion_prev_hour,Cluster
6,0.000000,0.0,18,6,16,54,3,89,0.000000,1
7,6.584959,0.0,18,7,17,70,4,95,0.000000,1
8,560.422022,0.0,18,8,18,72,3,100,6.584959,2
9,7720.582326,1.0,18,9,18,81,4,100,560.422022,2
10,9433.109309,2.0,19,10,18,89,4,93,7720.582326,2
...,...,...,...,...,...,...,...,...,...,...
18230,25399.000000,8.0,24,14,9,60,3,38,25598.000000,0
18231,25562.000000,5.0,25,15,8,43,3,34,25399.000000,0
18232,25386.000000,4.0,26,16,8,25,2,31,25562.000000,0
18233,22872.000000,2.0,26,17,8,18,1,32,25386.000000,0


In [44]:
datos_0 = datos_sinfecha[datos_sinfecha['Cluster'] == 0].drop(columns=['Cluster'])
datos_1 = datos_sinfecha[datos_sinfecha['Cluster'] == 1].drop(columns=['Cluster'])
datos_2 = datos_sinfecha[datos_sinfecha['Cluster'] == 2].drop(columns=['Cluster'])

In [45]:
X_0 = datos_0.drop('Generación', axis=1)
X_scaled_0 = scaler.fit_transform(X_0)
X_scaled_df_0 = pd.DataFrame(X_scaled_0, columns=X_0.columns)
X_0 = X_scaled_df_0.copy()
y_0 = datos_0['Generación']

In [46]:
X_1 = datos_1.drop('Generación', axis=1)
X_scaled_1 = scaler.fit_transform(X_1)
X_scaled_df_1 = pd.DataFrame(X_scaled_1, columns=X_1.columns)
X_1 = X_scaled_df_1.copy()
y_1 = datos_1['Generación']

In [47]:
X_2 = datos_2.drop('Generación', axis=1)
X_scaled_2 = scaler.fit_transform(X_2)
X_scaled_df_2 = pd.DataFrame(X_scaled_2, columns=X_2.columns)
X_2 = X_scaled_df_2.copy()
y_2 = datos_2['Generación']

In [48]:
#Porcentaje de entrenamiento
porcentaje = 0.8

#Horizonte de predicción
horizon = 1

#División de datos
lentrain100_0 = int(len(y_0) * porcentaje)
lentrain100_1 = int(len(y_1) * porcentaje)
lentrain100_2 = int(len(y_2) * porcentaje)

In [49]:
#Dividimos 80/20
X_train_0, X_test_0 = X_0.iloc[:lentrain100_0, :], X_0.iloc[lentrain100_0:, :]
y_train_0, y_test_0 = y_0.iloc[:lentrain100_0], y_0.iloc[lentrain100_0:]

X_train_1, X_test_1 = X_1.iloc[:lentrain100_1, :], X_1.iloc[lentrain100_1:, :]
y_train_1, y_test_1 = y_1.iloc[:lentrain100_1], y_1.iloc[lentrain100_1:]

X_train_2, X_test_2 = X_2.iloc[:lentrain100_2, :], X_2.iloc[lentrain100_2:, :]
y_train_2, y_test_2 = y_2.iloc[:lentrain100_2], y_2.iloc[lentrain100_2:]

In [50]:
X_train_0.shape

(4356, 8)

In [51]:
y_train_0.shape

(4356,)

In [52]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Conv1D, Bidirectional, GRU, Dense, Attention, Flatten, Dropout, MaxPooling1D, Input
from tensorflow.keras.models import Model

In [56]:
def build_qrkddn_model(input_shape):
    inputs = Input(shape=input_shape)
    
    # CNN Layer 1
    x = Conv1D(filters=64, kernel_size=4, activation='relu', padding='same')(inputs)
    
    # CNN Layer 2
    x = Conv1D(filters=128, kernel_size=4, activation='relu', padding='same')(x)
    
    # BiGRU Layer
    x = Bidirectional(GRU(128, return_sequences=True))(x)
    
    # Attention Layer
    attention = Attention()([x, x])
    
    # Flatten
    x = Flatten()(attention)
    
    # Fully Connected Layers
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.2)(x)
    outputs = Dense(1, activation='linear')(x)
    
    model = Model(inputs, outputs)
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model


In [57]:
X_train_3D_0 = np.expand_dims(X_train_0, axis=1)  # Reestructurar a (4356, 1, 8)

y_train_0 = np.array(y_train_0).reshape(-1, 1)  # Asegurar forma bidimensional

# Construir el modelo
model = build_qrkddn_model(input_shape=(1, X_train_3D_0.shape[2]))

# Entrenar el modelo
model.fit(X_train_3D_0, y_train_0, epochs=200, batch_size=128, validation_split=0.3)

Epoch 1/200


24/24 ━━━━━━━━━━━━━━━━━━━━ 40s 192ms/step - loss: 551075008.0000 - mae: 22170.6191 - val_loss: 484415648.0000 - val_mae: 20993.3008
Epoch 2/200
24/24 ━━━━━━━━━━━━━━━━━━━━ 2s 44ms/step - loss: 545276544.0000 - mae: 22030.6973 - val_loss: 480595008.0000 - val_mae: 20902.2109
Epoch 3/200
24/24 ━━━━━━━━━━━━━━━━━━━━ 2s 37ms/step - loss: 541784832.0000 - mae: 21960.4648 - val_loss: 476366496.0000 - val_mae: 20800.6035
Epoch 4/200
24/24 ━━━━━━━━━━━━━━━━━━━━ 2s 63ms/step - loss: 541827136.0000 - mae: 22016.3379 - val_loss: 471490368.0000 - val_mae: 20683.0469
Epoch 5/200
24/24 ━━━━━━━━━━━━━━━━━━━━ 2s 53ms/step - loss: 536534336.0000 - mae: 21851.3945 - val_loss: 465816160.0000 - val_mae: 20545.5723
Epoch 6/200
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 54ms/step - loss: 522890240.0000 - mae: 21508.3184 - val_loss: 459300896.0000 - val_mae: 20386.6367
Epoch 7/200
24/24 ━━━━━━━━━━━━━━━━━━━━ 2s 42ms/step - loss: 522563072.0000 - mae: 21547.4727 - val_loss: 451889280.0000 - val_mae: 20204.3203
Epoch 8/200
24/2

In [58]:
X_train_3D_1 = np.expand_dims(X_train_1, axis=1)  # Reestructurar a (4356, 1, 8)

y_train_1 = np.array(y_train_1).reshape(-1, 1)  # Asegurar forma bidimensional

# Construir el modelo
model_1 = build_qrkddn_model(input_shape=(1, X_train_3D_1.shape[2]))

# Entrenar el modelo
model_1.fit(X_train_3D_1, y_train_1, epochs=200, batch_size=128, validation_split=0.3)

Epoch 1/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 38s 734ms/step - loss: 1899217.7500 - mae: 507.6143 - val_loss: 3416241.2500 - val_mae: 429.3927
Epoch 2/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 107ms/step - loss: 1818507.6250 - mae: 537.5898 - val_loss: 3412603.7500 - val_mae: 429.8305
Epoch 3/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 99ms/step - loss: 1970598.0000 - mae: 531.5959 - val_loss: 3401627.2500 - val_mae: 431.2208
Epoch 4/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 3s 356ms/step - loss: 1707478.6250 - mae: 489.5388 - val_loss: 3382617.5000 - val_mae: 435.5179
Epoch 5/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 4s 96ms/step - loss: 1894662.0000 - mae: 525.5068 - val_loss: 3361002.0000 - val_mae: 445.3702
Epoch 6/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 155ms/step - loss: 1927880.1250 - mae: 562.2994 - val_loss: 3341336.2500 - val_mae: 458.7230
Epoch 7/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 93ms/step - loss: 1851727.6250 - mae: 557.3771 - val_loss: 3323726.2500 - val_mae: 472.4714
Epoch 8/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 58ms/step - loss: 1777023.2500 

In [59]:
X_train_3D_2 = np.expand_dims(X_train_2, axis=1)  # Reestructurar a (4356, 1, 8)

y_train_2 = np.array(y_train_2).reshape(-1, 1)  # Asegurar forma bidimensional

# Construir el modelo
model_2 = build_qrkddn_model(input_shape=(1, X_train_3D_2.shape[2]))

# Entrenar el modelo
model_2.fit(X_train_3D_2, y_train_2, epochs=200, batch_size=128, validation_split=0.3)

Epoch 1/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 34s 452ms/step - loss: 369142080.0000 - mae: 16843.5039 - val_loss: 348900640.0000 - val_mae: 17376.5547
Epoch 2/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 75ms/step - loss: 350952448.0000 - mae: 16326.9102 - val_loss: 348425664.0000 - val_mae: 17363.2402
Epoch 3/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 49ms/step - loss: 359545216.0000 - mae: 16465.1758 - val_loss: 347036544.0000 - val_mae: 17323.4551
Epoch 4/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 60ms/step - loss: 364849216.0000 - mae: 16668.1602 - val_loss: 345269920.0000 - val_mae: 17271.8730
Epoch 5/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 2s 116ms/step - loss: 362249376.0000 - mae: 16701.2676 - val_loss: 343536352.0000 - val_mae: 17221.4199
Epoch 6/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 68ms/step - loss: 354209472.0000 - mae: 16548.5723 - val_loss: 341713504.0000 - val_mae: 17168.4121
Epoch 7/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 55ms/step - loss: 351407616.0000 - mae: 16262.5088 - val_loss: 339723264.0000 - val_mae: 17110.3555
Epo

In [60]:
# Inicializar listas para métricas
resultados = pd.DataFrame(columns=['Hora', 'Real', 'Predicho'])
num_pruebas = len(X_test_0)
#Ciclo diario de predicción
for i in range(num_pruebas):
    inicio = i * horizon
    fin = inicio + horizon

    X_test_seg = X_test_0.iloc[inicio:fin, :]
    y_test_seg = y_test_0.iloc[inicio:fin]

    if len(X_test_seg) < horizon:
        break

    # 🔹 Añadir dimensión extra para que tenga la forma esperada
    X_test_seg_reshaped = np.expand_dims(X_test_seg, axis=1)  # (samples, 1, features)

    y_pred = model.predict(X_test_seg_reshaped)
    y_pred = np.clip(y_pred, 0, 30000)  # 🔹 Limitamos entre 0 y 30,000

    df_temp = pd.DataFrame({
        'Hora': X_test_seg.index,
        'Real': y_test_seg,
        'Predicho': y_pred.flatten()
    })

    resultados = pd.concat([resultados, df_temp], ignore_index=True)

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 201ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 482ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 154ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 190ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 422ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 173ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 191ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step   
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 872ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 689ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 211ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 768ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 386ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 253ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 358ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 301ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 344ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 176ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 493ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92m

In [61]:
# Inicializar listas para métricas
resultados_1 = pd.DataFrame(columns=['Hora', 'Real', 'Predicho'])
num_pruebas = len(X_test_1)
#Ciclo diario de predicción
for i in range(num_pruebas):
    inicio = i * horizon
    fin = inicio + horizon

    X_test_seg = X_test_1.iloc[inicio:fin, :]
    y_test_seg = y_test_1.iloc[inicio:fin]

    if len(X_test_seg) < horizon:
        break

    # 🔹 Añadir dimensión extra para que tenga la forma esperada
    X_test_seg_reshaped = np.expand_dims(X_test_seg, axis=1)  # (samples, 1, features)

    y_pred = model_1.predict(X_test_seg_reshaped)
    y_pred = np.clip(y_pred, 0, 30000)  # 🔹 Limitamos entre 0 y 30,000

    df_temp = pd.DataFrame({
        'Hora': X_test_seg.index,
        'Real': y_test_seg,
        'Predicho': y_pred.flatten()
    })

    resultados_1 = pd.concat([resultados_1, df_temp], ignore_index=True)

1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 182ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 315ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 807ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 764ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 166ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 307ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 154ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 221ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 493ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 232ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 162ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 776ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 529ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 545ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 262ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 169ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 489ms

In [62]:
# Inicializar listas para métricas
resultados_2 = pd.DataFrame(columns=['Hora', 'Real', 'Predicho'])
num_pruebas = len(X_test_2)
#Ciclo diario de predicción
for i in range(num_pruebas):
    inicio = i * horizon
    fin = inicio + horizon

    X_test_seg = X_test_2.iloc[inicio:fin, :]
    y_test_seg = y_test_2.iloc[inicio:fin]

    if len(X_test_seg) < horizon:
        break

    # 🔹 Añadir dimensión extra para que tenga la forma esperada
    X_test_seg_reshaped = np.expand_dims(X_test_seg, axis=1)  # (samples, 1, features)

    y_pred = model_2.predict(X_test_seg_reshaped)
    y_pred = np.clip(y_pred, 0, 30000)  # 🔹 Limitamos entre 0 y 30,000

    df_temp = pd.DataFrame({
        'Hora': X_test_seg.index,
        'Real': y_test_seg,
        'Predicho': y_pred.flatten()
    })

    resultados_2 = pd.concat([resultados_2, df_temp], ignore_index=True)

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 383ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 165ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 757ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 283ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 549ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 175ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 389ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 218ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 226ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 203ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 544ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 222ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 164ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 293ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 463ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 226ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 293ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 415ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 623ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 320m

In [67]:
from sklearn.metrics import mean_absolute_percentage_error as mean_absolute_percentage_error
from sklearn.metrics import mean_absolute_error as mean_absolute_error
from sklearn.metrics import mean_squared_error as mean_squared_error
from sklearn.metrics import r2_score as r2_score

In [68]:
print(f"MAE: {mean_absolute_error(resultados['Real'], resultados['Predicho']):.4f}")
print(f"RMSE: {np.sqrt(mean_squared_error(resultados['Real'], resultados['Predicho'])):.4f}")
print(f"R²: {r2_score(resultados['Real'], resultados['Predicho']):.4f}")

MAE: 2058.4035
RMSE: 3476.7710
R²: 0.6741


In [69]:
print(f"MAE: {mean_absolute_error(resultados_1['Real'], resultados_1['Predicho']):.4f}")
print(f"RMSE: {np.sqrt(mean_squared_error(resultados_1['Real'], resultados_1['Predicho'])):.4f}")
print(f"R²: {r2_score(resultados_1['Real'], resultados_1['Predicho']):.4f}")

MAE: 318.3512
RMSE: 604.4341
R²: 0.4603


In [70]:
print(f"MAE: {mean_absolute_error(resultados_2['Real'], resultados_2['Predicho']):.4f}")
print(f"RMSE: {np.sqrt(mean_squared_error(resultados_2['Real'], resultados_2['Predicho'])):.4f}")
print(f"R²: {r2_score(resultados_2['Real'], resultados_2['Predicho']):.4f}")

MAE: 2358.1355
RMSE: 3256.9296
R²: 0.7944


In [63]:
resultados

,Hora,Real,Predicho
0,4356,28469.0,28613.257812
1,4357,28107.0,28339.310547
2,4358,27998.0,28051.544922
3,4359,27794.0,28092.392578
4,4360,26996.0,27614.255859
...,...,...,...
1085,5441,25399.0,24545.521484
1086,5442,25562.0,24197.673828
1087,5443,25386.0,25049.886719
1088,5444,22872.0,24119.732422


In [64]:
resultados_1

,Hora,Real,Predicho
0,1440,1290.0,0.000000
1,1441,0.0,1.187121
2,1442,397.0,43.348034
3,1443,0.0,8.693435
4,1444,429.0,1241.097168
...,...,...,...
356,1796,71.0,0.924583
357,1797,0.0,1.316694
358,1798,71.0,1.874878
359,1799,0.0,1.755045


In [65]:
resultados_2

,Hora,Real,Predicho
0,2106,26701.0,24456.701172
1,2107,29243.0,26219.480469
2,2108,12684.0,7447.251953
3,2109,23118.0,23928.933594
4,2110,25770.0,26056.208984
...,...,...,...
522,2628,7356.0,7726.144531
523,2629,17638.0,19653.255859
524,2630,23339.0,22963.615234
525,2631,26323.0,25175.693359
